# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import openpyxl as Workbook

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2017, 8, 16)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,6/25/2017 12:00:00 AM,163.7,52.6,3.03005
1,6/26/2017 12:00:00 AM,202.4,42.5,4.15543
2,6/27/2017 12:00:00 AM,202.3,53.8,3.97314
3,6/28/2017 12:00:00 AM,197.7,52.4,3.93617
4,6/29/2017 12:00:00 AM,192.6,45.2,3.83577


In [5]:
# Calculate total miles driven, round values
voltDailyData['TotalMilesDriven'] = voltDailyData['MilesDriven'] + voltDailyData['EvMilesDriven']
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned,TotalMilesDriven
0,2017-06-25,163.7,52.6,3.03,216.3
1,2017-06-26,202.4,42.5,4.16,244.9
2,2017-06-27,202.3,53.8,3.97,256.1
3,2017-06-28,197.7,52.4,3.94,250.1
4,2017-06-29,192.6,45.2,3.84,237.8


In [6]:
#Volt Readings throughout the day (usually two)
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')

In [7]:
voltReadings.head()

,LocalTimestamp,BatteryLevel,ChargeState,OilLife,TirePressure_RF,TirePressure_RR,TirePressure_LF,TirePressure_LR,GallonsBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,8/16/2017 6:18:35 AM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,715.91140,65.61902,19876.81930,46977.40802,0.0,0.0,0.0,6.42466
1,8/15/2017 6:35:30 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,715.05284,65.64832,19874.26547,46942.02093,0.0,0.0,0.0,7.30171
2,8/14/2017 7:43:19 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,711.25933,65.72014,19813.34002,46744.06449,0.0,0.0,0.0,5.00078
3,8/14/2017 6:50:34 AM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,708.86329,65.73844,19748.53722,46599.56462,0.0,0.0,0.0,7.33870
4,8/13/2017 6:58:21 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,707.54243,65.78753,19745.32473,46547.46886,0.0,0.0,0.0,9.31206


In [8]:
# Sort based on timestamp, round values, convert object to timestamp 
voltReadings.sort_values(by='LifetimeMiles', inplace=True)
voltReadings['LocalTimestamp'] = pd.to_datetime(voltReadings['LocalTimestamp'])
voltReadings = voltReadings.round(2)
voltReadings.head()

,LocalTimestamp,BatteryLevel,ChargeState,OilLife,TirePressure_RF,TirePressure_RR,TirePressure_LF,TirePressure_LR,GallonsBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
103,2017-06-24 17:31:19,0.0,NaN,0.0,0.0,0.0,0.0,0.0,593.83,68.35,17941.91,40585.45,0.0,0.0,0.0,6.24
102,2017-06-25 05:35:50,0.0,NaN,0.0,0.0,0.0,0.0,0.0,593.83,68.35,17941.91,40585.45,0.0,0.0,0.0,6.24
101,2017-06-25 17:51:57,0.0,NaN,0.0,0.0,0.0,0.0,0.0,596.65,68.27,17988.50,40734.80,0.0,0.0,0.0,3.65
100,2017-06-26 05:52:46,0.0,NaN,0.0,0.0,0.0,0.0,0.0,596.86,68.27,17994.46,40749.16,0.0,0.0,0.0,3.32
99,2017-06-26 19:15:25,0.0,NaN,0.0,0.0,0.0,0.0,0.0,601.02,68.14,18036.93,40951.55,0.0,0.0,0.0,5.40


In [9]:
voltReadings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 103 to 0
Data columns (total 16 columns):
LocalTimestamp      104 non-null datetime64[ns]
BatteryLevel        104 non-null float64
ChargeState         0 non-null float64
OilLife             104 non-null float64
TirePressure_RF     104 non-null float64
TirePressure_RR     104 non-null float64
TirePressure_LF     104 non-null float64
TirePressure_LR     104 non-null float64
GallonsBurned       104 non-null float64
LifetimeFuelEcon    104 non-null float64
LifetimeEvMiles     104 non-null float64
LifetimeMiles       104 non-null float64
LastTripFuelEcon    104 non-null float64
LastTripEvMiles     104 non-null float64
LastTripMiles       104 non-null float64
GallonsOfGas        104 non-null float64
dtypes: datetime64[ns](1), float64(15)
memory usage: 13.8 KB


In [10]:
# Drop NA values and select columns desired
voltReadings = voltReadings.dropna(axis=1, how='any')
voltReadings = voltReadings[['LocalTimestamp', 'GallonsBurned', 'LifetimeFuelEcon', 'LifetimeMiles', 'GallonsOfGas']]
voltReadings.head()

,LocalTimestamp,GallonsBurned,LifetimeFuelEcon,LifetimeMiles,GallonsOfGas
103,2017-06-24 17:31:19,593.83,68.35,40585.45,6.24
102,2017-06-25 05:35:50,593.83,68.35,40585.45,6.24
101,2017-06-25 17:51:57,596.65,68.27,40734.80,3.65
100,2017-06-26 05:52:46,596.86,68.27,40749.16,3.32
99,2017-06-26 19:15:25,601.02,68.14,40951.55,5.40


In [11]:
voltReadings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 103 to 0
Data columns (total 5 columns):
LocalTimestamp      104 non-null datetime64[ns]
GallonsBurned       104 non-null float64
LifetimeFuelEcon    104 non-null float64
LifetimeMiles       104 non-null float64
GallonsOfGas        104 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 4.9 KB
